![Scala Intro](/notebooks/work/misc/thumbnail.jpg)

## Session Objectives

1. Grasp the **fundamentals of FP** by means of Scala
2. Learn the basics of **algebraic data types** (ADTs)
3. Understand the implications of having **functions as first-class citizens**
4. Get used to the **syntax** that simplifies dealing with functions

## So, What is Functional Programming?

Functional Programming is **"Programming With Pure Functions"**

([According to Wikipedia](https://en.wikipedia.org/wiki/Pure_function)) A pure function is a function that has the following properties:
1. Its return value is the same for the same arguments
2. Its evaluation has no side effects

#### Pure function example

#### Impure function example

#### Immutability

#### [Why Functional Programming Matters?](https://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf) (John Hughes)

> *"We conclude that since ***modularity*** is the key to successful programming, functional programming offers important advantages for software development."*

![Indiana Jones](/notebooks/work/misc/grial.jpg)

## Algebraic Data Types

* Make a data type to work with int lists

* We create a method `prepend` to add a new element in the front of the list

* Make a list instance and invoke the `prepend` method

### Case classes

* They provide common utilities automatically: shorter constructor, `toString` method, `copy` method, etc.

* We can use it to make list creation simpler and to enable reading instance contents

### Pattern Matching

* We can use this technique to compare a value against a sequence of patterns

* Pattern matching turns out to be really useful in combination with case classes (`sum` method)

* Invocation of `sum` method

## Lambda Expressions

* So far we've been working with methods (`incr` example)

* But functions are *first-class citizens* (ugly lambda expressions)

* We provide a new method, which `maps`s every element in the list

* Invocation of `map` method using `incr`

## Syntactic Sugar

* An operator is just a method (`concatenate` & `prepend`)

* It's possible to declare default arguments for parameters

* Variadic methods are great for certain situations like creating lists

* If you use `apply` as a method name, it's applied automatically

* Scala deploys syntax to simplify the creation of Lambda Expressions 

## Final Result

In [90]:
// IList is an Algebraic Data Type, made of Cons and End
sealed abstract class IList {
    
  // Operators are available
  def ::(h: Int): IList = Cons(h, this)
    
  // Pattern matching is very handy to deal with ADTs
  def sum: Int = this match {
    case Cons(h, t) => h + t.sum
    case End() => 0
  }
    
  // Functions can be passed as parameters
  // `map` is a higher order function
  def map(f: Int => Int): IList = this match {
    case Cons(h, t) => Cons(f(h), t.map(f))
    case End() => End()
  }
    
  def ++(other: IList): IList = this match {
    case Cons(h, t) => Cons(h, t ++ other)
    case End() => other
  }
}

// Case classes mitigate common boilerplate
// Default parameters could be helpful
case class Cons(val head: Int, val tail: IList = End()) extends IList
case class End() extends IList

object IList {
  // Variadic arguments are idoneous as a list creator
  def apply(xs: Int*): IList = {
    if (xs.isEmpty) End()
    else Cons(xs.head, apply(xs.tail:_*)) 
  }
}

defined class IList
defined class Cons
defined class End
defined object IList

## Takeaways

* Functional programming is programming with pure functions
* Algebraic data types are encoded as a "sum" of case clases
* Functions are treated as first-class citizens, which enables *higher order functions*
* Syntactic sugar is convenient to dulcify expressions

In [88]:
println("Thank You!")

Thank You!


![Applause](/notebooks/work/misc/applause.gif)